In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/Users/christinedekock/wp_internship/collaboration/')

In [17]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
import json

In [18]:
cd ../data/

/srv/home/christinedk/wp_internship/data


In [11]:
ls talk_history/

tmp_talk-advert-meta-info.json         tmp_talk-peacock-meta-info.json
tmp_talk-autobiography-meta-info.json  tmp_talk-weasel-meta-info.json
tmp_talk-fanpov-meta-info.json


In [12]:
with open("/srv/home/christinedk/wp_internship/data/talk_history/tmp_talk-autobiography-meta-info.json",'r') as my_file:
    text = [k for k in my_file.readlines()]

In [64]:
df = pd.read_json('/srv/home/christinedk/wp_internship/data/talk_history/tmp_talk-autobiography-meta-info.json',
             lines=True)

In [20]:
df.page_title.nunique()

4198

In [21]:
blp = pd.read_csv('/srv/home/christinedk/wp_internship/data/namespaces.csv')
blp = blp.query('page_namespace in (0,1)')

In [23]:
df.head()

,page_id,page_title,revision_id,revision_minor_edit,revision_text,revision_timestamp,user_id
0,44892193,Talk:Andy Bennett (musician),640972602,False,I notice that the Andy Bennett page is under d...,2015-01-04T18:14:08Z,6127189
1,44892193,Talk:Andy Bennett (musician),734817129,True,{{Old AfD multi|page=Andy Bennett (musician)|d...,2016-08-16T22:14:39Z,82835
2,44892193,Talk:Andy Bennett (musician),640261007,False,I notice that the Andy Bennett page is under d...,2014-12-30T17:08:28Z,-1
3,44892193,Talk:Andy Bennett (musician),643034413,False,{{Old AfD multi|page=Andy Bennett (musician)|d...,2015-01-18T10:48:10Z,1779282
4,44892193,Talk:Andy Bennett (musician),744179144,False,{{Old AfD multi|page=Andy Bennett (musician)|d...,2016-10-13T16:16:31Z,27015025


In [29]:
df.revision_text.values[2]

'I notice that the Andy Bennett page is under dispute. I do not understand why this is happening as Andy is not only a musician in his own right who has been in the band Sargant and The Elements but also has a solo career, very similar to other present and past members of Ocean Colour Scene (Simon Fowler, Steve Craddock, Damon Minchella etc).\n\nHaving started performing with the band in 1997 he went on to write, record and perform all over the world with the band\n\nHe is signed to a record label in his own right as Andy Bennett and continues to work with the members of the band OCS in projects etc.\n\nPlease can Wikipedia offer some guidance regarding any missing information and I will endeavour to add this. Alfie Mahone[[Special:Contributions/94.3.44.171|94.3.44.171]] ([[User talk:94.3.44.171|talk]]) 17:08, 30 December 2014 (UTC)'

In [31]:
df.revision_text.values[0].split('\n\n')

['I notice that the Andy Bennett page is under dispute. I do not understand why this is happening as Andy is not only a musician in his own right who has been in the band Sargant and The Elements but also has a solo career, very similar to other present and past members of Ocean Colour Scene (Simon Fowler, Steve Craddock, Damon Minchella etc).',
 'Having started performing with the band in 1997 he went on to write, record and perform all over the world with the band',
 'He is signed to a record label in his own right as Andy Bennett and continues to work with the members of the band OCS in projects etc.',
 'Please can Wikipedia offer some guidance regarding any missing information and I will endeavour to add this. Alfie Mahone[[Special:Contributions/94.3.44.171|94.3.44.171]] ([[User talk:94.3.44.171|talk]]) 17:08, 30 December 2014 (UTC)',
 "Alfie, you would be best to post at [[Wikipedia:Articles for deletion/Andy Bennett (musician)]], where the future of the page is being discussed. T

In [22]:
text_blp = [i for i in tqdm(text[1:]) if int(i[2]) in blp.page_id.values]

  0%|          | 0/82635 [00:00<?, ?it/s]

ValueError: invalid literal for int() with base 10: 'r'

In [21]:
revisions = pd.DataFrame(text_blp, columns=text[0])
revisions['page_namespace'] = revisions['page_namespace'].astype(int)
revisions = revisions.query('page_namespace in (0,1)')

In [30]:
def get_edits_pre_tag(data):
    ix = data['has_template'].idxmax()

    return data.loc[:ix]

In [31]:
revisions = revisions.sort_values(by=['page_id','revision_id'], ascending=True)
revisions['has_template'] = revisions['has_template'].fillna(0).astype(int)
pre_edit = revisions.groupby('page_id').apply(get_edits_pre_tag)
pre_edit.reset_index(drop=True, inplace=True)

In [32]:
revisions.groupby(['page_namespace'])['page_id'].count()

page_namespace
0    631111
1     67967
Name: page_id, dtype: int64

In [33]:
revisions.groupby(['page_namespace'])['page_id'].nunique()

page_namespace
0    3643
1    3643
Name: page_id, dtype: int64

In [34]:
# persist

In [35]:
revisions.to_csv('/srv/home/christinedk/wp_internship/data/autobiography.csv',index=False)

In [36]:
with open('//srv/home/christinedk/wp_internship/data/autobiography.json','w') as f:
    json.dump(revisions.to_dict(orient='records'),f)